## load our dependencies

In [1]:
import pandas as pd
import re

# load our dataframe

In [2]:
audible_path = './audible_uncleaned.csv'
audible_df = pd.read_csv(audible_path)

print("Audible Info")
print("*" * 45)
print(audible_df.info())
print("*" * 45)

print()

print("Audible Shape")
print("*" * 15)
print(audible_df.shape)
print("*" * 15)

Audible Info
*********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB
None
*********************************************

Audible Shape
***************
(87489, 8)
***************


# checking all unique Languages 🗺️ and dropping all but English and Spanish

In [3]:
# Check unique language values
unique_languages = audible_df['language'].unique()
print("Unique languages:", unique_languages)

# Filter the DataFrame to keep only English and Spanish entries
audible_df = audible_df[audible_df['language'].isin(['English', 'Spanish'])]


Unique languages: ['English' 'Hindi' 'spanish' 'german' 'french' 'catalan' 'swedish'
 'italian' 'danish' 'finnish' 'dutch' 'hebrew' 'russian' 'polish'
 'galician' 'afrikaans' 'icelandic' 'romanian' 'japanese' 'tamil'
 'portuguese' 'urdu' 'hungarian' 'czech' 'bulgarian' 'mandarin_chinese'
 'basque' 'korean' 'arabic' 'greek' 'turkish' 'ukrainian' 'slovene'
 'norwegian' 'telugu' 'lithuanian']


# extract info from 'stars'

##### '**4.5 out of 5 stars41 ratings**' ➡️ stars: 4.5 | ratings: 41 | rated: yes

##### '**Not rated yet**' ➡️ stars: NaN | ratings: NaN | rated: no 

In [4]:
# Rename Stars to use it with correct float values
audible_df.rename(columns={'stars': 'original_stars'}, inplace=True)

# Step 1: Extract star ratings as float
audible_df['stars'] = audible_df['original_stars'].str.extract(r'(\d+\.?\d*)').astype(float)

# Step 2: Extract rating counts as integers
audible_df['ratings'] = audible_df['original_stars'].str.extract(r'(\d+) ratings').astype(float)

# Step 3: Add 'rated' field
audible_df['rated'] = audible_df['original_stars'].apply(lambda x: 'no' if 'Not rated yet' in x else 'yes')

# Drop the original 'stars' column if it's no longer needed
audible_df.drop(columns=['original_stars'], inplace=True)

In [5]:
print("*" * 100)
print(audible_df[['stars', 'ratings', 'rated']].head())
print("*" * 100)

****************************************************************************************************
   stars  ratings rated
0    5.0     34.0   yes
1    4.5     41.0   yes
2    4.5     38.0   yes
3    4.5     12.0   yes
4    4.5    181.0   yes
****************************************************************************************************


# split 'releasedate' into more attributes

##### '**04-08-08**' ➡️ month: 04 | day: 08 | year: 08

In [19]:
# Split the 'releasedate' column into day, month, and year
audible_df[['day', 'month', 'year']] = audible_df['releasedate'].str.split('-', expand=True)

# Format the year as a four-digit year
audible_df['year'] = audible_df['year'].apply(lambda x: int(x) + 2000 if int(x) < 100 else int(x))

In [20]:
print("*" * 100)
print(audible_df[['day', 'month', 'year']].head())
print("*" * 100)

****************************************************************************************************
  day month  year
0  04    08  2008
1  01    05  2018
2  06    11  2020
3  05    10  2021
4  13    01  2010
****************************************************************************************************


# fix 'author' and 'narrator' to exclude _______by:

****some books may have multiple authors****

##### 'Writtenby:GeronimoStilton' ➡️ author: Geronimo Stilton 
##### 'Writtenby:RonaldL.Smith' ➡️ author: Ronald L. Smith 
##### 'Writtenby:HumzaArshad,HenryWhite' ➡️ author: Humza Arshad, Henry White 

##### 'Narratedby:BillLobely' ➡️ narrator: Bill Lobely 

In [8]:
# Remove 'Writtenby:' prefix and add spaces between names for 'author' field
audible_df['author'] = audible_df['author'].str.replace('Writtenby:', '', regex=False)
audible_df['author'] = audible_df['author'].apply(lambda x: re.sub(r'(\w)([A-Z])', r'\1 \2', ', '.join(x.split(','))))

# Remove 'Narratedby:' prefix and add spaces between names for 'narrator' field
audible_df['narrator'] = audible_df['narrator'].str.replace('Narratedby:', '', regex=False)
audible_df['narrator'] = audible_df['narrator'].apply(lambda x: re.sub(r'(\w)([A-Z])', r'\1 \2', x))


In [9]:
print("*" * 100)
print(audible_df[['author', 'narrator']].head())
print("*" * 100)

****************************************************************************************************
             author         narrator
0  Geronimo Stilton      Bill Lobely
1      Rick Riordan   Robbie Daymond
2       Jeff Kinney      Dan Russell
3      Rick Riordan  Soneela Nankani
4      Rick Riordan  Jesse Bernstein
****************************************************************************************************


# convert time into minutes for better use of units

##### '**time:11 hrs and 16 mins**' ➡️  minutes: 676

In [10]:
# Use lambda function to convert time to minutes
audible_df['minutes'] = audible_df['time'].apply(
    lambda x: (
        (int(re.search(r'(\d+)\s*hr', x).group(1)) * 60 if re.search(r'(\d+)\s*hr', x) else 0) +
        (int(re.search(r'(\d+)\s*min', x).group(1)) if re.search(r'(\d+)\s*min', x) else 0)
    )
)

In [11]:
print("*" * 100)
print(audible_df[['time']].head())
print("*" * 100)
print("*" * 100)
print(audible_df[['minutes']].head())
print("*" * 100)

****************************************************************************************************
                 time
0   2 hrs and 20 mins
1   13 hrs and 8 mins
2    2 hrs and 3 mins
3  11 hrs and 16 mins
4              10 hrs
****************************************************************************************************
****************************************************************************************************
   minutes
0      140
1      788
2      123
3      676
4      600
****************************************************************************************************


# convert price from INR to USD

*exchange rate from 04/11/2022 from when data was scraped*

##### **'price: 468'** ➡️ price_usd: xxxx

In [12]:
# Replace "Free" with "0" and remove commas
audible_df['price'] = audible_df['price'].replace("Free", "0").str.replace(',', '')

# Convert the 'price' column to numeric
audible_df['price'] = pd.to_numeric(audible_df['price'], errors='coerce')

# Use the historical exchange rate from April 11, 2022
historical_exchange_rate = 76.0  # 1 USD = 76 INR

# Convert the 'price' column from INR to USD
audible_df['price_usd'] = audible_df['price'] / historical_exchange_rate

# Round to two decimal places for clarity
audible_df['price_usd'] = audible_df['price_usd'].round(2)


In [13]:
print("*" * 100)
print(audible_df[['price', 'price_usd']].head())
print("*" * 100)

****************************************************************************************************
   price  price_usd
0  468.0       6.16
1  820.0      10.79
2  410.0       5.39
3  615.0       8.09
4  820.0      10.79
****************************************************************************************************


In [14]:
print("Final Table")
audible_df = audible_df[['name','author','narrator','time','minutes','releasedate','month', 'day', 'year','language','stars','rated','ratings','price','price_usd']]
audible_df['name'] = audible_df['name'].astype(str)
audible_df.to_csv('audible_output.csv', index=False)
print(audible_df.head())

Final Table
                                         name            author  \
0                  Geronimo Stilton #11 & #12  Geronimo Stilton   
1                            The Burning Maze      Rick Riordan   
2                                The Deep End       Jeff Kinney   
3                        Daughter of the Deep      Rick Riordan   
4  The Lightning Thief: Percy Jackson, Book 1      Rick Riordan   

          narrator                time  minutes releasedate month day  year  \
0      Bill Lobely   2 hrs and 20 mins      140    04-08-08    08  04  2008   
1   Robbie Daymond   13 hrs and 8 mins      788    01-05-18    05  01  2018   
2      Dan Russell    2 hrs and 3 mins      123    06-11-20    11  06  2020   
3  Soneela Nankani  11 hrs and 16 mins      676    05-10-21    10  05  2021   
4  Jesse Bernstein              10 hrs      600    13-01-10    01  13  2010   

  language  stars rated  ratings  price  price_usd  
0  English    5.0   yes     34.0  468.0       6.16  
1  E

In [15]:
print(audible_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 61884 entries, 0 to 87488
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         61884 non-null  object 
 1   author       61884 non-null  object 
 2   narrator     61884 non-null  object 
 3   time         61884 non-null  object 
 4   minutes      61884 non-null  int64  
 5   releasedate  61884 non-null  object 
 6   month        61884 non-null  object 
 7   day          61884 non-null  object 
 8   year         61884 non-null  int64  
 9   language     61884 non-null  object 
 10  stars        14590 non-null  float64
 11  rated        61884 non-null  object 
 12  ratings      8897 non-null   float64
 13  price        61884 non-null  float64
 14  price_usd    61884 non-null  float64
dtypes: float64(4), int64(2), object(9)
memory usage: 7.6+ MB
None


In [16]:

# Convert to string type
audible_df['language'] = audible_df['language'].astype(str)
audible_df['rated'] = audible_df['rated'].astype(str)

# Convert to integer type, handling non-numeric values if necessary
audible_df['minutes'] = pd.to_numeric(audible_df['minutes'], errors='coerce').fillna(0).astype(int)
audible_df['month'] = pd.to_numeric(audible_df['month'], errors='coerce').fillna(0).astype(int)
audible_df['day'] = pd.to_numeric(audible_df['day'], errors='coerce').fillna(0).astype(int)
audible_df['year'] = pd.to_numeric(audible_df['year'], errors='coerce').fillna(0).astype(int)
audible_df['ratings'] = pd.to_numeric(audible_df['ratings'], errors='coerce').fillna(0).astype(int)

# Convert remaining columns to float type
audible_df['stars'] = audible_df['stars'].astype(float)
audible_df['price'] = audible_df['price'].astype(float)
audible_df['price_usd'] = audible_df['price_usd'].astype(float)

# Verify final data types
print(audible_df.dtypes)

name            object
author          object
narrator        object
time            object
minutes          int64
releasedate     object
month            int64
day              int64
year             int64
language        object
stars          float64
rated           object
ratings          int64
price          float64
price_usd      float64
dtype: object


In [17]:
# Perform stratified sampling by language (10% sample size for each language group)
stratified_sample = audible_df.groupby('month', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

# Display the stratified sample
print("Stratified Sample by Month")
print(stratified_sample['month'].value_counts(normalize=True))  # Show proportion for each language in the sample
print(stratified_sample.head())


Stratified Sample by Month
month
3     0.105816
10    0.096769
2     0.092892
11    0.084330
12    0.082391
1     0.081745
9     0.081260
4     0.078675
8     0.076898
7     0.075283
5     0.073021
6     0.070921
Name: proportion, dtype: float64
                                                  name         author  \
461    Ivy & Bean Take Care of the Babysitter (Book 4)  Annie Barrows   
50684                                 The Winter Guest       W.C.Ryan   
60193                                  Chasing Secrets    Brandt Legg   
83373                                              Win  Vera Nazarian   
32267                                  Old Man Country  Thomas R.Cole   

               narrator                time  minutes releasedate  month  day  \
461      Hallie Ricardo    1 hr and 11 mins       71    04-01-22      1    4   
50684     Liam Hourican    9 hrs and 5 mins      545    06-01-22      1    6   
60193       Roger Wayne  22 hrs and 29 mins     1349    25-01-22      1   2

/var/folders/09/_ffptp312fj33qw8f8_zqqgc0000gn/T/ipykernel_15408/317001460.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = audible_df.groupby('month', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))


In [18]:
# Choose specific years as clusters, for example, 2020 and 2021
selected_years = [2000, 2005, 2010, 2015, 2020]

# Filter the dataset to include only the selected clusters (years)
cluster_sample = audible_df[audible_df['year'].isin(selected_years)]

# Display the cluster sample
print("Cluster Sample by Year")
print(cluster_sample['year'].value_counts())  # Show the distribution of selected years in the sample
print(cluster_sample.head())


Cluster Sample by Year
year
2020    7656
2015    2379
2010    1193
2005     330
2000     189
Name: count, dtype: int64
                                             name        author  \
2                                    The Deep End   Jeff Kinney   
4      The Lightning Thief: Percy Jackson, Book 1  Rick Riordan   
10       The Titan's Curse: Percy Jackson, Book 3  Rick Riordan   
20                              The Tower of Nero  Rick Riordan   
24  Percy Jackson and the Battle of the Labyrinth  Rick Riordan   

           narrator                time  minutes releasedate  month  day  \
2       Dan Russell    2 hrs and 3 mins      123    06-11-20     11    6   
4   Jesse Bernstein              10 hrs      600    13-01-10      1   13   
10  Jesse Bernstein   8 hrs and 48 mins      528    14-01-10      1   14   
20   Robbie Daymond  12 hrs and 12 mins      732    06-10-20     10    6   
24  Jesse Bernstein  10 hrs and 32 mins      632    21-01-10      1   21   

    year language  st